### Imports

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from PIL import Image
import pytesseract
import os
import time

### Configs

In [44]:
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'}
replace_dict = {' ': '+', ',': '%2C', "'": "%27"}
# liga_url = 'https://www.ligamagic.com.br/?view=cards%2Fsearch&card='
liga_url = 'https://www.ligamagic.com.br/?view=cards/card&card='
pytesseract.pytesseract.tesseract_cmd = 'E:/Tesseract/tesseract.exe'
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

### Functions

In [2]:
def get_html(url, headers):
    try:
        response = requests.get(url=url, headers=headers)
        response.status_code
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            # print("Informações recuperadas do site")
            return soup
        else:
            print(f"Falha em recuperar informações do Liga Magic. Status code: {response.status_code}")
            return False
    except Exception as e:
        print(f"Erro: {e}")
        return False

In [3]:
def download_image(url):
    try:
        # Send a GET request to the URL
        response = requests.get(url)
        
        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # Open the file in binary write mode and write the content of the response
            with open(url.split('/')[-1], 'wb') as f:
                f.write(response.content)
            # print("Imagem baixada com sucesso!")
            return True
        else:
            print(f"Falha ao baixar imagem. Status code: {response.status_code}")
            return False
    except Exception as e:
        print(f"Erro: {e}")
        return False

In [4]:
def render_image(image_path, x, y, width, height):
    try:
        # Open the image file
        image = Image.open(image_path)
        
        # Crop the image to the specified region
        cropped_image = image.crop((x, y, x + width, y + height))
        
        return cropped_image
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [5]:
def get_valor(digitos, padroes_valores):
    valor_card = ''
    for digito in digitos:
        # print(f'digito {digito}')
        if 'padrao' in digitos[digito].keys():
            encrypt = digitos[digito]['padrao']
            # print(f'\tpadroes {encrypt}')
            for en in encrypt:
                # print(f'\t\tpadrao {en}')
                if padroes_valores[en]['tipo'] == 'url':
                    image_path = padroes_valores[en]['valores'].split('/')[-1]
                    # print(f'\t\t\timagem {image_path}')
                elif padroes_valores[en]['tipo'] == 'background-position':
                    x = abs(int(padroes_valores[en]['valores'][0]))
                    y = abs(int(padroes_valores[en]['valores'][1]))
                    # print(f'\t\t\tx y {x} {y}')
                else:
                    width = abs(int(padroes_valores[en]['valores'][0]))
                    height = abs(int(padroes_valores[en]['valores'][1]))
                    # print(f'\t\t\tw h {width} {height}')
            cropped_image = render_image(image_path, x, y, width, height)
            if cropped_image:
                # print(f'\t\t\t\tdeu bao renderizar')
                digit_value = pytesseract.image_to_string(cropped_image, config='--psm 6 digits')
                digit_value = ''.join(filter(str.isdigit, digit_value))
                valor_card += f'{digit_value}'
        else:
            valor_card += f"{digitos[digito]['separador']}"
    return valor_card

In [6]:
def valor_card(digitos_div, soup):
    digitos = {}
    padroes = {}
    pos = 0
    # digitos
    for digito_div in digitos_div:
        if 'class' in digito_div.attrs:
            regx = re.findall('.*"([a-zA-Z0-9]+) ([a-zA-Z0-9]+) ([a-zA-Z0-9]+)".*', str(digito_div))
            # padrao
            if len(regx) > 0:
                digitos[pos] = {'padrao': list(regx[0])}
                for pattr in list(regx[0]):
                    style_value = re.findall('\.{busca}(\{{.*?\}})'.format(busca=pattr), str(soup.find('style')))[0]
                    if not pattr in padroes.keys():
                        padroes[pattr] = f'{style_value}'
        else: # virgula
            digitos[pos] = {'separador': '.'}
        pos += 1

    padroes_valores = {}
    for padrao in padroes:
        item = padroes[padrao]
        if 'url' in item:
            valor = re.findall('//(.*?)\)', item)[0]
            tipo = 'url'
        elif 'background-position' in item:
            positions = re.findall(':([\-\d]+)px ([\-\d]+)', item)
            valor = list(positions[0])
            tipo = 'background-position'
        else:
            positions = re.findall('width:([\-\d]+)px.*height:([\-\d]+)px', item)
            valor = list(positions[0])
            tipo = 'width-height'
        padroes_valores[padrao] = {'tipo':tipo,'valores':valor,'style': item}

    for padrao in padroes_valores:
        if padroes_valores[padrao]['tipo'] == 'url':
            i_url = f"https://{padroes_valores[padrao]['valores']}"
            if not os.path.exists(padroes_valores[padrao]['valores'].split('/')[-1]):
                download_image(i_url)

    preco = get_valor(digitos, padroes_valores)
    return preco

In [255]:
def get_card_info(card_name):
    output = []
    url_card = card_name.translate(card_name.maketrans(replace_dict))
    request_url = f'{liga_url}{url_card}'
    print(f'\tRecuperando ofertas para {card_name}')
    soup = get_html(request_url, headers)
    if soup:
        print(request_url)
        divs = soup.find_all('div', attrs={'mp': '1'})
        print(f'\t\tAnúncios encontrados: {len(divs)}')
        for oferta in range(len(divs)):
            # print(f'\t\t\tRecuperando informações da loja {oferta+1} de {len(divs)}')
            loja = divs[oferta]
            edicao = loja.find_all('div', class_="e-mob-edicao-lbl")[0].find('p').text.strip()
            preco_encoded = loja.find_all('div', class_="e-mob-preco")
            if len(preco_encoded[0].attrs['class']) == 1:
                digitos_div = preco_encoded[0].find_all('div')
                preco = valor_card(digitos_div, soup)
            else:
                try:
                    preco = re.findall('R\$ ([\d,]+)',loja.find_all('div', class_=' '.join(preco_encoded[0].attrs['class']))[0].text)[1].replace(',','.')
                except:
                    digitos_div = preco_encoded[0].find_all('div')
                    preco = valor_card(digitos_div, soup)
            texto = loja.find_all('div', class_="e-col4")[0].find('img').attrs['title']
            qualidade = loja.find_all('div', class_="e-col4")[0].find('font').text.strip()
            nm_loja = loja.find_all('div', class_="container-logo-selo")[0].find('img').attrs['title']
            unidades_div = loja.find_all('div', class_="e-col5")
            quantidade = valor_card(unidades_div, soup)
            try:
                tem_extra = loja.find_all('div', class_="e-col2")[0].find_all('div', class_='edicaoextras')[0].find_all('p', class_='extras')
                tipo = tem_extra[0].text.strip()
            except:
                tipo = 'Regular'
            try:
                loja_sus = loja.find_all('div', class_="container-logo-selo")[0].find('img').find('img').attrs['title']
            except:
                loja_sus = 'Loja não Verificada'
            results = {'card': card_name,
                       'loja': nm_loja,
                       'tipo_loja': loja_sus,
                       'edicao': edicao,
                       'tipo_card': tipo,
                       'preco': preco,
                       'idioma': texto,
                       'qualidade': qualidade,
                       'qtd_disponivel': quantidade
                       }
            output.append(results)
            # print(f'\t\t\t\t{results}')
        df = pd.DataFrame.from_dict(output)
        return df
    else:
        return None

### Cards

In [251]:
card_list = pd.read_csv('./card_list_rats.txt',header=0,sep='|')
card_list.drop_duplicates(ignore_index=True,inplace=True)

In [252]:
print(f'Quantidade de Cards: {card_list.shape[0]}')

Quantidade de Cards: 20


In [253]:
card_list

,QTD,CARD_NAME
0,1,"Karumonix, the Rat King"
1,1,Typhoid Rats
2,1,Tangled Colony
3,1,Lord Skitter's Blessing
4,1,Lord Skitter's Butcher
5,1,"Lord Skitter, Sewer King"
6,1,Echoing Return
7,1,Fatal Push
8,1,Cabal Stronghold
9,1,Faceless Haven


### Loop Execucao

In [256]:
print('Consultando lista de cards no Liga Magic')
dfs = []
for idx in range(card_list.shape[0]):
    card_name = card_list.iloc[idx]['CARD_NAME']
    card_qtd = card_list.iloc[idx]['QTD']
    dfs.append(get_card_info(card_name))            

Consultando lista de cards no Liga Magic
	Recuperando ofertas para Karumonix, the Rat King
https://www.ligamagic.com.br/?view=cards/card&card=Karumonix%2C+the+Rat+King
		Anúncios encontrados: 286
	Recuperando ofertas para Typhoid Rats
https://www.ligamagic.com.br/?view=cards/card&card=Typhoid+Rats
		Anúncios encontrados: 465
	Recuperando ofertas para Tangled Colony
https://www.ligamagic.com.br/?view=cards/card&card=Tangled+Colony
		Anúncios encontrados: 214
	Recuperando ofertas para Lord Skitter's Blessing
https://www.ligamagic.com.br/?view=cards/card&card=Lord+Skitter%27s+Blessing
		Anúncios encontrados: 173
	Recuperando ofertas para Lord Skitter's Butcher
https://www.ligamagic.com.br/?view=cards/card&card=Lord+Skitter%27s+Butcher
		Anúncios encontrados: 172
	Recuperando ofertas para Lord Skitter, Sewer King
https://www.ligamagic.com.br/?view=cards/card&card=Lord+Skitter%2C+Sewer+King
		Anúncios encontrados: 84
	Recuperando ofertas para Echoing Return
https://www.ligamagic.com.br/?vie

In [257]:
len(dfs)

20

In [276]:
dfs_agrupados = []
dfs_filtrados = []
dfs_transpose = []
for df in dfs:
    df_filtro = df.loc[((df['qualidade'] == 'NM') | (df['qualidade'] == 'M'))& (df['tipo_loja'] != 'Loja não Verificada')]
    dfs_filtrados.append(df_filtro[['card','loja','edicao','tipo_card','idioma','preco','qtd_disponivel']])
    df_filtrado = df_filtro.copy()
    df_filtrado = df_filtrado.groupby(['card','loja','edicao','tipo_card','idioma','preco'],as_index=False).sum()
    df_filtrado['preco(disponibilidade)'] = df_filtrado.apply(lambda x: f'{x.preco} ({x.qtd_disponivel})', axis=1)
    dfs_agrupados.append(df_filtrado[['loja','card','edicao','tipo_card','idioma','preco(disponibilidade)']])
    # dfs_transpose.append(df_filtrado[['card','loja','edicao','tipo_card','idioma','preco(disponibilidade)']].drop_duplicates().pivot(index=['card','edicao','tipo_card','idioma'], columns='loja', values='preco(disponibilidade)').fillna(0))
    # dfs_transpose.append(df_filtro[['card','loja','edicao','tipo_card','idioma','preco']].drop_duplicates().pivot(index=['card','edicao','tipo_card','idioma'], columns='loja', values='preco').fillna(0))
    

In [260]:
card_list

,QTD,CARD_NAME
0,1,"Karumonix, the Rat King"
1,1,Typhoid Rats
2,1,Tangled Colony
3,1,Lord Skitter's Blessing
4,1,Lord Skitter's Butcher
5,1,"Lord Skitter, Sewer King"
6,1,Echoing Return
7,1,Fatal Push
8,1,Cabal Stronghold
9,1,Faceless Haven


In [277]:
for i in range(len(dfs)):
    df = dfs[i]
    if df.shape[0]>0:
        print(df.iloc[i])

card                                        Karumonix, the Rat King
loja                                           Armageddon Card Shop
tipo_loja         Loja Verificada - Possui loja física aberta ao...
edicao                                       Phyrexia: Tudo será um
tipo_card                                                      Foil
preco                                                          0.83
idioma                                                       Inglês
qualidade                                                         D
qtd_disponivel                                                    1
Name: 0, dtype: object
card                                                   Typhoid Rats
loja                                                 Gold House MTG
tipo_loja         Loja Verificada - Possui loja física aberta ao...
edicao                                            Destino Reescrito
tipo_card                                                   Regular
preco                    

In [249]:
pd.concat(dfs_filtrados,axis=0,ignore_index=False).fillna(0).to_csv('empilhados.csv')

In [250]:
pd.concat(dfs_transpose,axis=0,ignore_index=False).fillna(0).to_csv('transpostos.csv')